<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Concurrency-with-Shared-Variables" data-toc-modified-id="Concurrency-with-Shared-Variables-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Concurrency with Shared Variables</a></span><ul class="toc-item"><li><span><a href="#Race-Conditions" data-toc-modified-id="Race-Conditions-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Race Conditions</a></span></li><li><span><a href="#Mutual-Exclusion:-sync.Mutex" data-toc-modified-id="Mutual-Exclusion:-sync.Mutex-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Mutual Exclusion: sync.Mutex</a></span></li><li><span><a href="#Read/Write-Mutexes:-sync.RWMutex" data-toc-modified-id="Read/Write-Mutexes:-sync.RWMutex-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Read/Write Mutexes: sync.RWMutex</a></span></li><li><span><a href="#Memory-Synchronization" data-toc-modified-id="Memory-Synchronization-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Memory Synchronization</a></span></li><li><span><a href="#Lazy-Initialization:-sync.Once" data-toc-modified-id="Lazy-Initialization:-sync.Once-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Lazy Initialization: sync.Once</a></span></li><li><span><a href="#The-Race-Detector" data-toc-modified-id="The-Race-Detector-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span>The Race Detector</a></span></li><li><span><a href="#Example:-Concurrent-Non-Blocking-Cache" data-toc-modified-id="Example:-Concurrent-Non-Blocking-Cache-9.7"><span class="toc-item-num">9.7&nbsp;&nbsp;</span>Example: Concurrent Non-Blocking Cache</a></span><ul class="toc-item"><li><span><a href="#memo2-with-sync.Mutex,-but-with-no-performance-gains" data-toc-modified-id="memo2-with-sync.Mutex,-but-with-no-performance-gains-9.7.1"><span class="toc-item-num">9.7.1&nbsp;&nbsp;</span>memo2 with sync.Mutex, but with no performance gains</a></span></li></ul></li><li><span><a href="#Goroutines-and-Threads" data-toc-modified-id="Goroutines-and-Threads-9.8"><span class="toc-item-num">9.8&nbsp;&nbsp;</span>Goroutines and Threads</a></span><ul class="toc-item"><li><span><a href="#Growable-Stacks" data-toc-modified-id="Growable-Stacks-9.8.1"><span class="toc-item-num">9.8.1&nbsp;&nbsp;</span>Growable Stacks</a></span></li><li><span><a href="#Groroutine-Scheduling" data-toc-modified-id="Groroutine-Scheduling-9.8.2"><span class="toc-item-num">9.8.2&nbsp;&nbsp;</span>Groroutine Scheduling</a></span></li><li><span><a href="#GOMAXPROCS" data-toc-modified-id="GOMAXPROCS-9.8.3"><span class="toc-item-num">9.8.3&nbsp;&nbsp;</span>GOMAXPROCS</a></span></li><li><span><a href="#Goroutines-Have-No-Identify" data-toc-modified-id="Goroutines-Have-No-Identify-9.8.4"><span class="toc-item-num">9.8.4&nbsp;&nbsp;</span>Goroutines Have No Identify</a></span></li></ul></li></ul></li></ul></div>

# Concurrency with Shared Variables

## Race Conditions

In [57]:
var balance int
func Deposit(amount int) {  balance = balance + amount }
func Balance() int {return  balance }

In [105]:
// Alice:
import ("time";"fmt")
%%
//go Deposit(100)
//go Deposit(200)
go func() {
    Deposit(100)
    fmt.Println("in B =", Balance()) 
}()

//Alice
go func() {
    Deposit(200)                  // A1
    fmt.Println("=", Balance())   // A2
}()
//Bob:

//go func() {
//    Deposit(100)                  //B
//    fmt.Println("in B =", Balance()) 
//}()

time.Sleep(100* time.Millisecond)//delay termination
//fmt.Println("Final After sleep =", Balance()) 

in B = 100
= 300


In [106]:
%%
for i:=0; i<100; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(4)
       // Deposit(1)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 400
= 400
= 388
= 400
= 400
= 400
= 400
= 380
= 400
= 400
= 400
= 356
= 376
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 388
= 400
= 400
= 376
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 372
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 392
= 400
= 388
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 396
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 392
= 400
= 400
= 400
= 400


In [197]:
%%
var x []int

go func() { x = make([]int,1000000)}()
go func() { x = make([]int,10)}()
time.Sleep(1* time.Millisecond)
x[999999] = 1 // Note: undefined behavior; memory corruption possible

* A data race occurs whenever two goroutines access the same variable concurrently and at least one of the accesses is a write
* Three ways to avoid a data race
  * The first way: not to write the variable
  * The second way: avoid acessing the variable from multiple goroutines
    * Do not communicate by sharing memory; share memory by communicating
  * The third way: allow many goroutines to access the variable, but only one at a time
    * mutual exclusion

In [ ]:
// The first way:not to write the variable
// There is a data race accessing the map
import "image"
var icons = make(map[string]image.Image)
func loadIcon(name string) image.Image
// not concurrency-safe
func Icon(name string) image.Image {
    icon, ok := icons[name]
    if !ok {
        icon = loadIcon(name)
        icons[name] = icon
    }
    return icon
}

In [ ]:
// initialize the map with all necessary entries before creating goroutines and never modify again
// any goroutings may safely call Icon concurrently, only reads the map
import "image"
var icons = map[string]image.Image {
    "spades.png":  loadIcon("spades.png"),
    "hearts.png":  loadIcon("hearts.png"),
    "diamonds.png":loadIcon("diamonds.png"),
    "clubs.png":   loadIcon("clubs.png"),
}
// Concurrency-safe,
func Icon(name string) image.Image {return icons[name]}

* The second way
  * Use a channel to send the confining goroutine a request to query or update the variable
    * monitor goroutine: broker access to a confined variable using channel requests 

In [107]:
//gopl.io/ch9/bank1
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 261.
//!+

// Package bank provides a concurrency-safe bank with one account.
//package bank

var deposits = make(chan int) // send amount to deposit
var balances = make(chan int) // receive balance

func Deposit(amount int) { deposits <- amount }
func Balance() int       { return <-balances }

func teller() {
	var balance int // balance is confined to teller goroutine
	for {
		select {
		case amount := <-deposits:
			balance += amount
		case balances <- balance:
		}
	}
}

func init() {
	go teller() // start the monitor goroutine
}

//!-

In [108]:
//back_test
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

//package bank_test

import (
	"fmt"
	"testing"

	"gopl.io/ch9/bank1"
)

func TestBank(t *testing.T) {
	done := make(chan struct{})

	// Alice
	go func() {
		bank.Deposit(200)
		fmt.Println("=", bank.Balance())
		done <- struct{}{}
	}()

	// Bob
	go func() {
		bank.Deposit(100)
		done <- struct{}{}
	}()

	// Wait for both transactions.
	<-done
	<-done

	if got, want := bank.Balance(), 300; got != want {
		t.Errorf("Balance = %d, want %d", got, want)
	}
}

In [117]:
%%
var t testing.T
TestBank(&t)

= 300


In [131]:
import (
	"fmt"
	"testing"
	"gopl.io/ch9/bank1"
)
%%
done := make(chan struct{})
for j :=0; j<200; j++ {
    go func() {
        bank.Deposit(4)
       // Deposit(1)
        done <- struct{}{}
    }()
}
for i:= 0; i <100; i++ {
    <-done
}
//time.Sleep(100* time.Millisecond)
fmt.Println("=",bank.Balance())


= 800


In [199]:
import "fmt"
type Cake struct {state string}
func baker(cooked chan <- *Cake) {
    for {
      cake := new(Cake)
      cake.state = "cooked"
      fmt.Println(cake.state)
      cooked <- cake // baker never touches this cake again
    }
}

func icer(iced chan <- *Cake, cooked <- chan *Cake){
    for cake := range cooked {
        cake.state = "iced"
        fmt.Println(cake.state)
        iced <- cake //icer never touches this cake again
    }
}

In [204]:
// Cakes are serially confied, first to the baker, then to the icer
import "time"
%%
cooked := make(chan *Cake,10)
iced := make(chan *Cake,20)
go icer(iced, cooked)
go baker(cooked)
time.Sleep(5* time.Millisecond)

cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
iced
iced
iced
iced
iced
iced
iced
iced
iced
iced
iced
iced
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked
iced
iced
iced
iced
iced
iced
iced
iced
iced
cooked
cooked
cooked
cooked
cooked
cooked
cooked
cooked


## Mutual Exclusion: sync.Mutex

In [165]:
//gopl.io/ch9/bank2
var (
    sema = make (chan struct{},1)
    balance int
)
func Deposit(amount int) {
    sema <- struct{}{} // acquire token
    balance = balance + amount
    <- sema  //release token
}
func Balance() int {
    //sema <- struct{}{} // acquire token
    b := balance
    //<- sema // release token
    return b
}

In [166]:
%%
done := make(chan struct{})
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      //go Deposit(4)
        go func() {
          Deposit(4)
       // Deposit(1)
          done <- struct{}{}
        }()
    }
    for i:= 0; i <100; i++ {
      <-done
    }
    //time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400


In [205]:
//gopl.io/ch9/bank3
import "sync"
var (
  mu sync.Mutex //guards balance
  balance int
)

func Deposit(amount int) {
    mu.Lock()
    balance = balance + amount
    mu.Unlock()
}
func Balance() int {
    mu.Lock()
    b := balance
    mu.Unlock()
    return b
}

In [155]:
import ("time";"fmt")
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(4)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400


In [207]:
func Balance() int {
    mu.Lock()
    defer mu.Unlock()
    return balance
}

* Withdraw function, three implementation
  * without lock
  * lock a second time and lead to a deadlock
  * internal implementation without second lock

In [208]:
// Not Atomic
func Withdraw(amount int) bool {
    Deposit(-amount)
    if Balance() < 0 {
        Deposit(amount)
        return false // insufficient funds
    }
    return true
}

In [214]:
import ("time";"fmt")
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0;j<100; j++ {
        go Deposit(4)
        go Withdraw(2)
    }
    time.Sleep(200* time.Millisecond)
    fmt.Println("=",Balance())
}

= 202
= 202
= 202
= 202
= 202
= 202
= 202
= 202
= 200
= 200


In [215]:
// incorrect, may be a deadlock
func Withdraw(amount int) bool {
    mu.Lock()
    defer mu.Unlock()
    Deposit(-amount) 
    if Balance() < 0 {
        Deposit(amount)
        return false // insufficient funds
    }
    return true
}

In [217]:
import ("time";"fmt")
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(4)
      go Withdraw(2)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

fatal error: all goroutines are asleep - deadlock!

goroutine 1 [semacquire]:
sync.runtime_SemacquireMutex(0x589dc49f?, 0xc0?, 0x140000a7eb8?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Balance()
	 [[ Cell [207] Line 2 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:23 +0x94
main.main()
	 [[ Cell [217] Line 10 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:114 +0x74

goroutine 17 [select]:
gopl.io/ch9/bank1.teller()
	/Users/skhuang/course/go2023/pkg/mod/gopl.io@v0.0.0-20211004154805-1ae3ec64947b/ch9/bank1/bank.go:19 +0x78
created by gopl.io/ch9/bank1.init.0
	/Users/skhuang/course/go2023/pkg/mod/gopl.io@v0.0.0-20211004154805-1ae3ec64947b/ch9/bank1/bank.go:28 +0x24

goroutine 19 [select]:
main.teller(

	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	 [[ Cell [217] Line 7 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 32 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4

sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	 [[ Cell [217] Line 7 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 52 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg2

goroutine 65 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	 [[ Cell [217] Line 7 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 66 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	 [[ Cell [205] Line 9 

	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:110 +0x114

goroutine 79 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	 [[ Cell [217] Line 7 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 80 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*M

...

goroutine 135 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	/var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 136 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg20

main.Deposit(0x4?)
	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:110 +0x114

goroutine 149 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	/var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 150 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex

created by main.main
	 [[ Cell [217] Line 7 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 176 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:110 +0x114

goroutine 177 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sy

goroutine 190 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	/var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:110 +0x114

goroutine 191 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg20

sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Withdraw(0x2?)
	 [[ Cell [215] Line 3 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:60 +0x98
created by main.main
	 [[ Cell [217] Line 7 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:111 +0x120

goroutine 204 [semacquire]:
sync.runtime_SemacquireMutex(0x0?, 0x0?, 0x0?)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/sema.go:77 +0x24
sync.(*Mutex).lockSlow(0x102d088b0)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:171 +0x178
sync.(*Mutex).Lock(...)
	/opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/sync/mutex.go:90
main.Deposit(0x4?)
	 [[ Cell [205] Line 9 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:29 +0x88
created by main.main
	 [[ Cell [217] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_4772149a/main.go:110 +0x114

goroutine 205 [semacquire]:
sync.runtime_Semac

In [218]:
import "sync"
var (
  mu sync.Mutex //guards balance
  balance int
)
func Withdraw(amount int) bool {
    mu.Lock()
    defer mu.Unlock()
    deposit(-amount)
    if balance < 0 {
        deposit(amount)
        return false // insufficient funds
    }
    return true
}
func Deposit(amount int) {
    mu.Lock()
    defer mu.Unlock()
    deposit(amount)
}
func Balance() int {
    mu.Lock()
    defer mu.Unlock()
    return balance
}
// require the lock be held
func deposit(amount int) { balance += amount}

In [220]:
import ("time";"fmt")
import "sync"
var (
  mu sync.Mutex //guards balance
  balance int
)
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(4)
      go Withdraw(1)
    }
    time.Sleep(1500* time.Millisecond)
    fmt.Println("=",Balance())
}

= 300
= 300
= 301
= 301
= 301
= 301
= 301
= 301
= 301
= 301


## Read/Write Mutexes: sync.RWMutex
* Multiple readers, single writer lock, provided with sync.RWMutex

In [221]:
import "sync"
var (
    mu sync.RWMutex
    balance int
)
func Deposit(amount int) {
    mu.Lock()
    balance = balance + amount
    mu.Unlock()
}

func Balance()int {
    mu.RLock()  // readers lock
    defer mu.RUnlock()
    return balance
}

In [225]:
import ("time";"fmt")
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(2)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 200
= 200
= 200
= 200
= 200
= 200
= 200
= 200
= 200
= 200


## Memory Synchronization

In [234]:
%%
var x, y int

for i:=0; i< 20; i++ {
 x, y = 0,0
 go func(){
   x = 1                 // A1
   fmt.Print("y:",y," ") // A2
 }()

 go func(){
    y = 1                //B1
    fmt.Print("x:",x," ")//B2
 }()
 time.Sleep(300 * time.Millisecond)
 fmt.Println("")
 
}
time.Sleep(500 * time.Millisecond)


x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 


## Lazy Initialization: sync.Once
* Defer an expensive initialization step until the moment it is needed

In [ ]:
//Lazy initialization 
import "image"
var icons map[string]image.Image
func loadIcon(string)image.Image
func loadIcons(){
    icons = map[string]image.Image {
        "spades.png": loadIcon("spades.png"),
        "hearts.png": loadIcon("hearts.png"),
        "diamonds.png":loadIcon("diamonds.png"),
        "clubs.png":   loadIcon("clubs.png"),
    }
}
// Not concurrency-safe
func Icon(name string) image.Image {
    if icons == nil {
        loadIcons() // one-time initialization
    }
    return icons[name]
}

In [ ]:
//icons may not be nil, but the initialization is not complete
import "image"
var icons map[string]image.Image
func loadIcons() {
    icons = make(map[string]image.Image)
    icons["spades.png"] = loadIcon("spades.png")
    icons["hearts.png"] = loadIcon("hearts.png")
    icons["diamonds.png"]=loadIcon("diamonds.png")
    icons["clubs.png"] = loadIcon("clubs.png")
}

In [ ]:
// concurrency safe, but expensive for only one goroutine can access Icon
import "sync"
import "image"
var mu sync.Mutex // guard Icons
var icons map[string]image.Image
func Icon(name string)image.Image {
    mu.Lock()
    defer mu.Unlock()
    if icons == nil {
        loadIcons()
    }
    return icons[name]
}

In [ ]:
var mu sync.RWMutex // guards icons
var icons map[string]image.Image
// Concurrency-safe
func Icon(name string) image.Image {
    mu.RLock()
    if icons != nil {
        icon := icons[name]
        mu.RUnlock()
        return icon
    }
    mu.RUnlock()
    //acquire an exclusive lock
    mu.Lock()
    if icons == nil {
        loadIcons()
    }
    icon := icons[name]
    mu.Unlock()
    return icon
}

In [ ]:
var loadIconsOnce sync.Once
var icons map[string]image.Image
//Concurrency-safe
func Icon(name string) image.Image {
    loadIconsOnce.Do(loadIcons)
    return icons[name]
}

## The Race Detector

* Go runtime and toolchain are equipped with a dynamic analysis tool, the race detector
* Add the -race flat to go build, go run, or go test command
  * Record all accesses to shared variables that occurred during execution, along with the identify of the goroutinge that read or wrote the variable
  * Record all synchronization events
    * go statements, channel operations, and calls to (*sync.Mutex).Lock, (*sync.WaitGroup).wait, and so on.

## Example: Concurrent Non-Blocking Cache

In [236]:
// Package memo  1 provides a concurrency-unsafe
// memoization of a function of type Func.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 272.

//!+


//package memo

// A Memo caches the results of calling a Func.
type Memo struct {
	f     Func
	cache map[string]result
}

// Func is the type of the function to memoize.
type Func func(key string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]result)}
}

// NOTE: not concurrency-safe!
func (memo *Memo) Get(key string) (interface{}, error) {
	res, ok := memo.cache[key]
	if !ok {
		res.value, res.err = memo.f(key)
		memo.cache[key] = res
	}
	return res.value, res.err
}

//!-

In [237]:
!go test -run=TestConcurrent -race -v gopl.io/ch9/memo1

=== RUN   TestConcurrent
https://godoc.org, 506.641708ms, 32378 bytes
https://godoc.org, 510.141125ms, 32378 bytes
https://golang.org, 552.716417ms, 61870 bytes
https://golang.org, 559.70625ms, 61870 bytes
https://play.golang.org, 916.501833ms, 30053 bytes
https://play.golang.org, 929.173167ms, 30053 bytes
http://gopl.io, 1.252338875s, 4154 bytes
Write at 0x00c00011eed0 by goroutine 16:
  runtime.mapaccess2_faststr()
      /opt/homebrew/Cellar/go@1.19/1.19.13/libexec/src/runtime/map_faststr.go:108 +0x43c
  gopl.io/ch9/memo1.(*Memo).Get()
      /Users/skhuang/course/go2023/pkg/mod/gopl.io@v0.0.0-20211004154805-1ae3ec64947b/ch9/memo1/memo.go:35 +0xdc
  gopl.io/ch9/memotest.Concurrent.func1()
      /Users/skhuang/course/go2023/pkg/mod/gopl.io@v0.0.0-20211004154805-1ae3ec64947b/ch9/memotest/memotest.go:93 +0x9c
  gopl.io/ch9/memotest.Concurrent.func2()
      /Users/skhuang/course/go2023/pkg/mod/gopl.io@v0.0.0-20211004154805-1ae3ec64947b/ch9/memotest/memotest.go:100 +0x54

Previous write at

exit status 1


### memo2 with sync.Mutex, but with no performance gains

In [ ]:
// Package memo  2 provides a concurrency-safe memoization a function of
// type Func.  Concurrent requests are serialized by a Mutex.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 275.


//package memo

import "sync"

// Func is the type of the function to memoize.
type Func func(string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]result)}
}

//!+

type Memo struct {
	f     Func
	mu    sync.Mutex // guards cache
	cache map[string]result
}

// Get is concurrency-safe.
func (memo *Memo) Get(key string) (value interface{}, err error) {
	memo.mu.Lock() 
	res, ok := memo.cache[key]
	if !ok {
		res.value, res.err = memo.f(key)
		memo.cache[key] = res
	}
	memo.mu.Unlock()
	return res.value, res.err
}

//!-

In [249]:
!go test -run=TestConcurrent -race -v gopl.io/ch9/memo2


=== RUN   TestConcurrent
https://golang.org, 504.677292ms, 61870 bytes
https://play.golang.org, 1.777028667s, 30053 bytes
http://gopl.io, 2.842549125s, 4154 bytes
https://godoc.org, 3.453561875s, 32378 bytes
https://golang.org, 3.45326675s, 61870 bytes
https://godoc.org, 3.453329875s, 32378 bytes
https://play.golang.org, 3.453219125s, 30053 bytes
http://gopl.io, 3.453237s, 4154 bytes
--- PASS: TestConcurrent (3.45s)
PASS
ok  	gopl.io/ch9/memo2	(cached)


In [251]:
// Package memo 3 provides a concurrency-safe memoization a function of
// type Func.  Requests for different keys run concurrently.
// Concurrent requests for the same key result in duplicate work.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 276.


//package memo

import "sync"

type Memo struct {
	f     Func
	mu    sync.Mutex // guards cache
	cache map[string]result
}

type Func func(string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]result)}
}

//!+

func (memo *Memo) Get(key string) (value interface{}, err error) {
	memo.mu.Lock()
	res, ok := memo.cache[key]
	memo.mu.Unlock()
	if !ok {
		res.value, res.err = memo.f(key)

		// Between the two critical sections, several goroutines
		// may race to compute f(key) and update the map.
		memo.mu.Lock()
		memo.cache[key] = res
		memo.mu.Unlock()
	}
	return res.value, res.err
}

//!-

In [252]:
!go test -run=TestConcurrent -race -v gopl.io/ch9/memo3


=== RUN   TestConcurrent
https://godoc.org, 468.860125ms, 32378 bytes
https://godoc.org, 475.465792ms, 32378 bytes
https://golang.org, 494.28875ms, 61870 bytes
https://golang.org, 548.106708ms, 61870 bytes
http://gopl.io, 900.935583ms, 4154 bytes
https://play.golang.org, 902.671125ms, 30053 bytes
http://gopl.io, 909.889917ms, 4154 bytes
https://play.golang.org, 942.65975ms, 30053 bytes
--- PASS: TestConcurrent (0.95s)
PASS
ok  	gopl.io/ch9/memo3	(cached)


In [253]:
// Package memo 4 provides a concurrency-safe memoization a function of
// a function.  Requests for different keys proceed in parallel.
// Concurrent requests for the same key block until the first completes.
// This implementation uses a Mutex.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 276.

//package memo
type Func func(string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

//
type entry struct {
	res   result
	ready chan struct{} // closed when res is ready
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]*entry)}
}

type Memo struct {
	f     Func
	mu    sync.Mutex // guards cache
	cache map[string]*entry
}

func (memo *Memo) Get(key string) (value interface{}, err error) {
	memo.mu.Lock()
	e := memo.cache[key]
	if e == nil {
		// This is the first request for this key.
		// This goroutine becomes responsible for computing
		// the value and broadcasting the ready condition.
		e = &entry{ready: make(chan struct{})}
		memo.cache[key] = e
		memo.mu.Unlock()

		e.res.value, e.res.err = memo.f(key)

		close(e.ready) // broadcast ready condition
	} else {
		// This is a repeat request for this key.
		memo.mu.Unlock()

		<-e.ready // wait for ready condition
	}
	return e.res.value, e.res.err
}

//!-

In [254]:
!go test -run=TestConcurrent -race -v gopl.io/ch9/memo4


=== RUN   TestConcurrent
https://godoc.org, 498.568541ms, 32378 bytes
https://godoc.org, 498.048958ms, 32378 bytes
https://golang.org, 606.961583ms, 61870 bytes
https://golang.org, 606.672125ms, 61870 bytes
https://play.golang.org, 903.206041ms, 30053 bytes
https://play.golang.org, 902.523583ms, 30053 bytes
http://gopl.io, 931.218ms, 4154 bytes
http://gopl.io, 931.130709ms, 4154 bytes
--- PASS: TestConcurrent (0.93s)
PASS
ok  	gopl.io/ch9/memo4	(cached)


In [255]:
// Package memo 5 provides a concurrency-safe non-blocking memoization
// of a function.  Requests for different keys proceed in parallel.
// Concurrent requests for the same key block until the first completes.
// This implementation uses a monitor goroutine.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 278.

//package memo

//!+Func

// Func is the type of the function to memoize.
type Func func(key string) (interface{}, error)

// A result is the result of calling a Func.
type result struct {
	value interface{}
	err   error
}

type entry struct {
	res   result
	ready chan struct{} // closed when res is ready
}

//!-Func

//!+get

// A request is a message requesting that the Func be applied to key.
type request struct {
	key      string
	response chan<- result // the client wants a single result
}

type Memo struct{ requests chan request }

// New returns a memoization of f.  Clients must subsequently call Close.
func New(f Func) *Memo {
	memo := &Memo{requests: make(chan request)}
	go memo.server(f)
	return memo
}

func (memo *Memo) Get(key string) (interface{}, error) {
	response := make(chan result)
	memo.requests <- request{key, response}
	res := <-response
	return res.value, res.err
}

func (memo *Memo) Close() { close(memo.requests) }

//!-get

//!+monitor

func (memo *Memo) server(f Func) {
	cache := make(map[string]*entry)
	for req := range memo.requests {
		e := cache[req.key]
		if e == nil {
			// This is the first request for this key.
			e = &entry{ready: make(chan struct{})}
			cache[req.key] = e
			go e.call(f, req.key) // call f(key)
		}
		go e.deliver(req.response)
	}
}

func (e *entry) call(f Func, key string) {
	// Evaluate the function.
	e.res.value, e.res.err = f(key)
	// Broadcast the ready condition.
	close(e.ready)
}

func (e *entry) deliver(response chan<- result) {
	// Wait for the ready condition.
	<-e.ready
	// Send the result to the client.
	response <- e.res
}

//!-monitor

In [257]:
!go test -run=TestConcurrent -race -v gopl.io/ch9/memo5


=== RUN   TestConcurrent
https://godoc.org, 498.090333ms, 32378 bytes
https://godoc.org, 498.446291ms, 32378 bytes
https://play.golang.org, 1.001095458s, 30053 bytes
https://play.golang.org, 1.001764916s, 30053 bytes
http://gopl.io, 1.289401875s, 4154 bytes
http://gopl.io, 1.289167625s, 4154 bytes
https://golang.org, 1.502951292s, 61870 bytes
https://golang.org, 1.503326875s, 61870 bytes
--- PASS: TestConcurrent (1.50s)
PASS
ok  	gopl.io/ch9/memo5	(cached)


## Goroutines and Threads

### Growable Stacks
* a goroutine start with a small stack, typically 2KB
* a goroutine's stack is not fixed
  * it grows and shrinks as needed, with the limit of 1GB

### Groroutine Scheduling
* The Go runtime contains its own scheduler using a technique known as m:n scheduling
    * it multiplexes (or schedules) m go routines on n OS threads
    * the Go scheduler is not invoked periodically by a hardware timer
      * implicitly by certain Go language constructs
        * time.Sleep or blocks in a channel or mutex operation

### GOMAXPROCS
* GOMAXPROCS is the n in m:n scheduling

In [266]:
import "fmt"
%%
for i:=1;i<100;i++ {
    go fmt.Print(0)
    fmt.Print(1)
}
fmt.Println()
// GOMAXPROC=1 go run hacker-cliche.go
// GOMAXPROC=2 go run hacker-cliche.go

111111111010011001110000000011111111111010000000010000111110000110001111111111100100001110000010100001111111100111110000000001111110000000000001111111001110000111111111101111110000000011


In [268]:
!pwd
!GOMAXPROC=1 go run hacker-cliche.go
!GOMAXPROC=2 go run hacker-cliche.go
!GOMAXPROC=3 go run hacker-cliche.go
!GOMAXPROC=4 go run hacker-cliche.go

/Users/skhuang/course/go2023/go2023/ch9
11111111111111111111111010111000000000000001011111111111111111111111111111111111001000000000000000010000000000000100000000100000000011100001110000111111011111111100000011111111111
11111101111111001000000000000111111111111100000100000000111111111001111111110011100000000110000000001111000000111111111100000000111110000000111110000011111111001000000111111111111111
0111111111111111111111111111000000000000111111111111111111111111110100000000000000000111111100000000000000000011111111000000011111000000000001111111100000111111100000001111111100000000011
0111111111111111111111110100010000000000000000000100111110011111000001111111100100011100000000100010111011000111111000001111111100000000000011111110000011101111111000001111111111000000111


### Goroutines Have No Identify
* Goroutines have no notion of identity that is accessible to the programmer
* Go encourages a simpler style of programming in which parameters that affect the behavior of a function are explicit 
